# Premium Allocation Tool

In [1]:
import numpy as np
import pandas as pd
import json
import uuid  

from pat_sql import PatJob
from pat_common import PatFlag, AppSettings

# Get job list 


## Delete Job

In [6]:
# pd.DataFrame(json.loads(PatJob.get_job_list()))
lst= [a for a in range(100000150, 100000170)]
PatJob.delete(*lst)

,job_id,job_guid,job_name,receive_time,update_time,status,user_name,user_email
0,100000066,f76371b2-40c0-41de-bb3e-876bb61d7c59 ...,Chubb_2021_MAP_try,2021-10-24T19:33:10.648260,2021-10-24T19:40:26.967012,finished,Chengyou,chengyou.xiao@guycarp.com
1,100000049,d1bfecd3-bf6b-456c-95f1-4e6ce23fa75c ...,Chubb_2021_MAP_try,2021-10-24T19:07:56.029022,2021-10-24T19:13:58.324780,finished,Chengyou,chengyou.xiao@guycarp.com
2,100000027,fefed966-519e-46aa-9451-00d47e347099 ...,PAT_Test_AIG_with_correction,2021-10-24T03:12:07.614655,2021-10-24T03:16:28.131710,finished,cxiao,chengyou.xiao@guycarp.com
3,100000009,5085872a-385f-4303-89b5-ddd513d3ba73 ...,PAT_Test_AIG,2021-10-24T02:59:22.573403,2021-10-24T03:00:33.169961,finished,cxiao,chengyou.xiao@guycarp.com


## Job parameters

In [ ]:
ret = PatJob.get_job_para(100000197)
if ret:
    print(json.dumps(json.loads(ret),indent=4))


## Result summary 

In [ ]:
df = PatJob.get_summary(100000197)
df

## Results

In [ ]:
df = PatJob.get_results(100000197)
df

## Validation data

In [ ]:
df1,df2,df3 = PatJob.get_validation_data(100000197)
print(len(df1), len(df2), len(df3))

In [ ]:
df2.sort_values(by='LocationIDStack')

# Create New Job 

In [ ]:
job_para = json.load(open('data/pat_job_test.json'))
job_para['job_guid']=str(uuid.uuid4())

job = PatJob(job_para)
job.job_id

# Process

In [ ]:
job.perform_analysis()

# Testing

In [ ]:

import json
import numpy as np
import pandas as pd

import pyodbc
from bcpandas import SqlCreds, to_sql

df = pd.read_csv('data/psold_aoi.csv')

creds = SqlCreds(AppSettings.PAT_JOB_SVR, AppSettings.PAT_JOB_DB, AppSettings.PAT_JOB_USR, AppSettings.PAT_JOB_PWD)
to_sql(df,'psold_aoi', creds, index=False, if_exists='append') 

In [ ]:
job_conn = f'''DRIVER={{SQL Server}};Server={AppSettings.PAT_JOB_SVR};Database={AppSettings.PAT_JOB_DB};
            User Id={AppSettings.PAT_JOB_USR};Password={AppSettings.PAT_JOB_PWD};
            MultipleActiveResultSets=true;'''
with pyodbc.connect(job_conn) as conn:
             dfHPR = pd.read_sql_query(f"""select * from psold_hpr order by limit""", conn)
dfHPR

In [ ]:
select LocationIDStack,
	sum(case when job_id = 100000126 then premium else 0 end) as prem_with_spider,
	sum(case when job_id = 100000127 then premium else 0 end) as prem_wo_spider
from pat_premium
where job_id  in (100000126, 100000127)
group by LocationIDStack